In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
worst_initial_timepoint = "2012-10-01"
worst_future_timepoint = "2013-10-01"

best_initial_timepoint = "2016-01-01"
best_future_timepoint = "2017-01-01"

In [ ]:
df = pd.read_csv(
    "../results/clade_frequencies_for_h3n2.tsv",
    sep="\t",
    parse_dates=[
        "timepoint",
        "future_timepoint",
    ],
)

In [ ]:
df.head()

In [ ]:
worst_df = df.loc[
    (
        (df["timepoint"] == worst_initial_timepoint) &
        (df["future_timepoint"] == worst_future_timepoint) &
        (df["delay_type"].isin(["ideal", "realistic"]))
    ),
    :,
].copy()

In [ ]:
worst_df.shape

In [ ]:
worst_df.sort_values("absolute_forecast_error", ascending=False)

In [ ]:
worst_df.query("frequency >= 0.1")

In [ ]:
worst_df.query("clade_membership == 'A.6.8.5.2.4.1'")

In [ ]:
worst_df.groupby("delay_type")["absolute_forecast_error"].sum()

In [ ]:
worst_df.query("frequency >= 0.1").groupby("delay_type")["absolute_forecast_error"].sum()

In [ ]:
worst_df.query("frequency < 0.1").groupby("delay_type")["absolute_forecast_error"].sum()

In [ ]:
worst_realistic_df = worst_df.query("delay_type == 'realistic'").copy()

In [ ]:
worst_ideal_df = worst_df.query("delay_type == 'ideal'").copy()

In [ ]:
worst_ideal_df

In [ ]:
def sum_child_clades(dataframe, clade, sum_fields, group_by):
    clade_dataframe = dataframe.loc[
        (
            (dataframe["clade_membership"] == clade) |
            (dataframe["clade_membership"].str.startswith(f"{clade}."))
        ),
        sum_fields + group_by,
    ].groupby(group_by).sum().reset_index()
    clade_dataframe["clade_membership"] = clade
    
    return clade_dataframe

In [ ]:
sum_fields = [
    "frequency",
    "projected_frequency",
    "n_strains",
    "observed_frequency",
    "future_n_strains",
]

In [ ]:
group_by = [
    "sample",
    "timepoint",
    "future_timepoint",
    "delta_month",
    "delay_type",
]

In [ ]:
# Should return a data frame with frequencies and counts grouped for the given clade
# with frequencies all at 100%, since A.6 is the MRCA.
sum_child_clades(worst_ideal_df, "A.6", sum_fields=sum_fields, group_by=group_by)

In [ ]:
sum_child_clades(worst_ideal_df, "A.6.8.5.2.4.1", sum_fields=sum_fields, group_by=group_by)

In [ ]:
sum_child_clades(worst_realistic_df, "A.6.8.5.2.4.1", sum_fields=sum_fields, group_by=group_by)

In [ ]:
collapsed_dfs = []
for keys, grouped_df in df.groupby(group_by):
    print(keys)
    collapsed_grouped_dfs = []
    for clade in grouped_df["clade_membership"].drop_duplicates().values:
        collapsed_grouped_dfs.append(
            sum_child_clades(
                grouped_df,
                clade,
                sum_fields,
                group_by,
            )
        )
    collapsed_dfs.append(
        pd.concat(
            collapsed_grouped_dfs,
            ignore_index=True,
        )
    )

collapsed_df = pd.concat(
    collapsed_dfs,
    ignore_index=True,
)

In [ ]:
collapsed_df

In [ ]:
df.shape

In [ ]:
collapsed_df["forecast_error"] = collapsed_df["observed_frequency"] - collapsed_df["projected_frequency"]

In [ ]:
collapsed_df["absolute_forecast_error"] = collapsed_df["forecast_error"].abs()

In [ ]:
worst_df = collapsed_df.loc[
    (
        (collapsed_df["timepoint"] == worst_initial_timepoint) &
        (collapsed_df["future_timepoint"] == worst_future_timepoint) &
        (collapsed_df["delay_type"].isin(["ideal", "realistic"]))
    ),
    :,
].copy()

In [ ]:
worst_df

In [ ]:
worst_df.groupby("delay_type")["absolute_forecast_error"].sum()